In [1]:
import os
from GetDataLoaders import get_dataloaders, get_short_dataloaders
from architectures.AlexNetFeature import AlexNetFeature
from architectures.NonLinearClassifier import Flatten, Classifier
import torch
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from torch import nn
import time
from torch import optim
from torch.nn import functional as F
from tqdm import tqdm

In [2]:
'''
# we skip the probs for now
gama = 2.0
with open(os.path.join("./PUprobs", 'prob.dat'), 'r') as file_input:
    train_prob_str = file_input.readlines()
    train_prob = [float(i_prob_str.rstrip('\n')) for i_prob_str in train_prob_str]
    print(len(train_prob)/4.0)
    train_weight = [1.0 if 0==i%4 else 1-train_prob[i]**gama for i in range(len(train_prob))]
'''

'\n# we skip the probs for now\ngama = 2.0\nwith open(os.path.join("./PUprobs", \'prob.dat\'), \'r\') as file_input:\n    train_prob_str = file_input.readlines()\n    train_prob = [float(i_prob_str.rstrip(\'\n\')) for i_prob_str in train_prob_str]\n    print(len(train_prob)/4.0)\n    train_weight = [1.0 if 0==i%4 else 1-train_prob[i]**gama for i in range(len(train_prob))]\n'

In [3]:
use_cuda = torch.cuda.is_available()
device = "cuda" if use_cuda else "cpu"
batch_size = 192
lr = 0.1
LUT_lr = [(5, 0.01),(25, 0.002),(45, 0.0004),(65,0.00008)]
num_epochs = 65
momentum = 0.9
weight_decay = 5e-4
nesterov = True
num_classes = 200
loaders = get_dataloaders('imagenet', batch_size=batch_size, num_workers=2, unsupervised=False, simclr=False)

In [4]:
feature_net = AlexNetFeature().to(device)
#load pretrained weights in feature_net
state_dict = torch.load("weights/AlexNet_Decoupling_200.pth")
feature_net.load_state_dict(state_dict['feature_net'])

feature_net.eval()
for param in feature_net.parameters():
    param.requires_grad = False

classifier_net = Classifier().to(device)
classifier_optimizer = optim.SGD(classifier_net.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=nesterov)
Networks =   {'classifier':classifier_net}
Optimizers = {'classifier':classifier_optimizer}

Criterions = {'CE': nn.CrossEntropyLoss()}

In [5]:
def adjust_learning_rates(epoch):
    # filter out the networks that are not trainable and that do
    # not have a learning rate Look Up Table (LUT_lr) in their optim_params
    lr = next((lr for (max_epoch, lr) in LUT_lr if max_epoch>epoch), LUT_lr[-1][1])
    for key in Optimizers:
        for g in Optimizers[key].param_groups:
            prev_lr = g['lr']
            if prev_lr != lr:
                print("Learning Rate Updated from {} to {}".format(prev_lr, lr))
                g['lr'] = lr

def train_step(batch, train=True):
    data, targets = batch
    
    if train is True:
        for key in Optimizers:
            Optimizers[key].zero_grad()
    
    #to cuda
    
    data, targets = data.to(device), targets.to(device)
   
    
    #collect features
    with torch.no_grad():
        features = feature_net(data, ['conv5'])
    
    if train is False:
        with torch.no_grad():
            pred = Networks['classifier'](features)
            #calculate loss
            loss_cls =  Criterions['CE'](pred, targets)
            
    else:
        pred = Networks['classifier'](features)
        #calculate loss
        loss_cls =  Criterions['CE'](pred, targets)
    
    if train is True:
        loss_cls.backward()
        for key in Optimizers:
            Optimizers[key].step()
    
    #calculate classification accuracy
    pred = F.softmax(pred, dim=1)
    pred = pred.argmax(dim=1, keepdim=True)
    correct = pred.eq(targets.view_as(pred)).sum().item()
   
    
    return loss_cls.item(), correct
    

In [6]:
def train_validate(data_loader, epoch, train=True):
    mode = "Train" if train else "Valid"
    if train is True:
        for key in Networks:
            Networks[key].train()
    else:
        for key in Networks:
            Networks[key].eval()
    
    losses = []
    correct = 0
    
    if train:
        adjust_learning_rates(epoch)
    
    start_time = time.time()
    
    tqdm_bar = tqdm(data_loader)
    total_number = 0
    for batch_idx, sample in enumerate(tqdm_bar):
        
        loss, correct_step = train_step(sample, train=train)
        losses.append(loss)
        correct += correct_step
        total_number += sample[0].size(0)
        tqdm_bar.set_description('{} Epoch: {} Loss: {:.6f}, Accuracy: {}/{} [{:.4f}%]'.format(mode, epoch, loss, correct, total_number, 100.0*(correct/total_number)))
    
    end_time = time.time()
    print("Time for epoch pass {}".format(end_time-start_time))
    
    loss = float(np.mean(losses))
    acc =  float(correct / len(data_loader.dataset))
    print('{} set: Average loss: {:.4f}, Accuracy:{}/{} ({:.4f}%)\n'.format(mode, loss, correct, len(data_loader.dataset), 100.0*acc))
    return loss, acc


def run_main_loop(loaders, num_epochs):
    writer = SummaryWriter('logs/Imagenet-Classification-Without-Finetuning-basemethod-NonLinear')
    save_path = "weights/imagenetclassifierwithoutfinetune_base_nonlinear.pth"
    best_acc = 0
    for epoch in range(num_epochs):
        #print("Performing {}th epoch".format(epoch))
        
        train_loss, train_acc = train_validate(loaders['train_loader'], epoch, train=True)
        val_loss, val_acc = train_validate(loaders['valid_loader'], epoch, train=False)
    
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/Valid', val_loss, epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Accuracy/Valid', val_acc, epoch)
    
        
        if val_acc > best_acc  :
            best_acc = val_acc
            #save model
            states = {
                'epoch': epoch + 1,
                'best_accuracy': best_acc
            }
            for key in Networks:
                states[key+"model"] = Networks[key].state_dict()
            for key in Optimizers:
                states[key+"optimizer"] = Optimizers[key].state_dict()
            torch.save(states, save_path)
            print('Model Saved')

In [7]:
run_main_loop(loaders, num_epochs)

  0%|          | 0/417 [00:00<?, ?it/s]

Learning Rate Updated from 0.1 to 0.01


Train Epoch: 0 Loss: 4.342133, Accuracy: 16449/80000 [20.5613%]: 100%|██████████| 417/417 [01:42<00:00,  4.07it/s]
  0%|          | 0/53 [00:00<?, ?it/s]

Time for epoch pass 102.50038886070251
Train set: Average loss: 5.8361, Accuracy:16449/80000 (20.5613%)



Valid Epoch: 0 Loss: 4.454844, Accuracy: 2730/10000 [27.3000%]: 100%|██████████| 53/53 [00:11<00:00,  4.58it/s]


...

Time for epoch pass 9.877525329589844
Valid set: Average loss: 3.1739, Accuracy:3739/10000 (37.3900%)



Train Epoch: 64 Loss: 0.097344, Accuracy: 79481/80000 [99.3513%]: 100%|██████████| 417/417 [01:28<00:00,  4.72it/s]
  0%|          | 0/53 [00:00<?, ?it/s]

Time for epoch pass 88.34890151023865
Train set: Average loss: 0.0963, Accuracy:79481/80000 (99.3513%)



Valid Epoch: 64 Loss: 3.591559, Accuracy: 3723/10000 [37.2300%]: 100%|██████████| 53/53 [00:09<00:00,  5.45it/s]

Time for epoch pass 9.735226154327393
Valid set: Average loss: 3.1723, Accuracy:3723/10000 (37.2300%)



In [8]:
loss, acc = train_validate(loaders['test_loader'], 1, train=False)

Valid Epoch: 1 Loss: 2.787955, Accuracy: 7444/20000 [37.2200%]: 100%|██████████| 105/105 [00:22<00:00,  4.65it/s]

Time for epoch pass 22.587539196014404
Valid set: Average loss: 3.1463, Accuracy:7444/20000 (37.2200%)

